In [2]:
import pandas as pd
import numpy as np
from src import soporte as sp

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Fase 1: Exploración y Limpieza
## 1. Exploración Inicial:

In [4]:
df_flight = pd.read_csv("data/Customer Flight Activity.csv")
df_flight.head()

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0
3,100214,2017,1,0,0,0,0,0.0,0,0
4,100272,2017,1,0,0,0,0,0.0,0,0


In [5]:
df_loyalty = pd.read_csv("data/Customer Loyalty History.csv")
df_loyalty.head()

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,NaN,NaN


In [4]:
print(f"El csv de Actividad de Vuelos de los Clientes tiene {df_flight.shape[0]} filas y {df_flight.shape[1]} columnas ")
print("--------------")
print(f"El csv de Historial de los Clientes tiene {df_loyalty.shape[0]} filas y {df_loyalty.shape[1]} columnas ")

El csv de Actividad de Vuelos de los Clientes tiene 405624 filas y 10 columnas 
--------------
El csv de Historial de los Clientes tiene 16737 filas y 16 columnas 


In [18]:
df_loyalty["Salary"].value_counts()

Salary
 101933.0    23
 62283.0     14
 61809.0     14
 51573.0     14
 64001.0     13
             ..
 95907.0      1
 72440.0      1
 88633.0      1
 100572.0     1
-57297.0      1
Name: count, Length: 5890, dtype: int64

In [19]:
df_loyalty.loc[df_loyalty["Salary"] == -57297.0]

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
16735,906428,Canada,Yukon,Whitehorse,Y2K 6R0,Male,Bachelor,-57297.0,Married,Star,10018.66,2018 Promotion,2018,4,<NA>,<NA>


In [20]:
df_loyalty["Education"].value_counts()

Education
Bachelor                10475
College                  4238
High School or Below      782
Doctor                    734
Master                    508
Name: count, dtype: int64

In [51]:
df_loyalty[df_loyalty["Education"] == "High School or Below"]["Salary"].describe()

count      782.000000
mean     61199.161125
std      12105.354128
min      21853.000000
25%      52151.250000
50%      61915.000000
75%      71384.500000
max      93875.000000
Name: Salary, dtype: float64

In [32]:
df_loyalty["Salary"].isnull().sum()

np.int64(4238)

In [46]:
bachelor = df_loyalty[df_loyalty["Education"] == "Bachelor"]

In [47]:
bachelor[bachelor["Province"] == "Yukon"]["Salary"].describe()

count        68.000000
mean      68814.250000
std       17474.762024
min       31456.000000
25%       55676.750000
50%       63301.000000
75%       82026.750000
max      104966.000000
Name: Salary, dtype: float64

In [48]:
df_loyalty.groupby("Education")["Salary"].count()

Education
Bachelor                10475
College                     0
Doctor                    734
High School or Below      782
Master                    508
Name: Salary, dtype: int64

In [ ]:
#El salario negativo lo cambio a positivo porque es un valor que se corresponde con la zona

In [6]:
df_loyalty["Salary"] = abs(df_loyalty["Salary"])

In [49]:
sp.exploracion(df_flight)

El DataFrame tiene 405624 filas y 10 columnas.
    Tiene 1864 datos duplicados, lo que supone un porcentaje de 0.0% de los datos.

    Hay 0 columnas con datos nulos, y son: 
    []

    y sin nulos hay 10 columnas y son: 
    ['Loyalty Number', 'Year', 'Month', 'Flights Booked', 'Flights with Companions', 'Total Flights', 'Distance', 'Points Accumulated', 'Points Redeemed', 'Dollar Cost Points Redeemed']


    A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
Loyalty Number,0.0%,100.0%,int64,16737
Year,0.0%,100.0%,int64,2
Month,0.0%,100.0%,int64,12
Flights Booked,0.0%,100.0%,int64,22
Flights with Companions,0.0%,100.0%,int64,12


Principales estadísticos de las columnas categóricas:
No hay columnas categóricas en el DataFrame.
Principales estadísticos de las columnas numéricas:


,count,mean,std,min,25%,50%,75%,max
Loyalty Number,405624.0,550037.873084,258935.286969,100018.0,326961.00,550834.0,772194.00,999986.0
Year,405624.0,2017.500000,0.500001,2017.0,2017.00,2017.5,2018.00,2018.0
Month,405624.0,6.500000,3.452057,1.0,3.75,6.5,9.25,12.0
Flights Booked,405624.0,4.115052,5.225518,0.0,0.00,1.0,8.00,21.0
Flights with Companions,405624.0,1.031805,2.076869,0.0,0.00,0.0,1.00,11.0
Total Flights,405624.0,5.146858,6.521227,0.0,0.00,1.0,10.00,32.0
Distance,405624.0,1208.880059,1433.155320,0.0,0.00,488.0,2336.00,6293.0
Points Accumulated,405624.0,123.692721,146.599831,0.0,0.00,50.0,239.00,676.5
Points Redeemed,405624.0,30.696872,125.486049,0.0,0.00,0.0,0.00,876.0
Dollar Cost Points Redeemed,405624.0,2.484503,10.150038,0.0,0.00,0.0,0.00,71.0


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
Loyalty Number,0.0%,100.0%,int64,16737
Year,0.0%,100.0%,int64,2
Month,0.0%,100.0%,int64,12
Flights Booked,0.0%,100.0%,int64,22
Flights with Companions,0.0%,100.0%,int64,12
Total Flights,0.0%,100.0%,int64,33
Distance,0.0%,100.0%,int64,4746
Points Accumulated,0.0%,100.0%,float64,1549
Points Redeemed,0.0%,100.0%,int64,587
Dollar Cost Points Redeemed,0.0%,100.0%,int64,49


In [50]:
sp.exploracion(df_loyalty)


El DataFrame tiene 16737 filas y 16 columnas.
    Tiene 0 datos duplicados, lo que supone un porcentaje de 0.0% de los datos.

    Hay 3 columnas con datos nulos, y son: 
    ['Salary', 'Cancellation Year', 'Cancellation Month']

    y sin nulos hay 13 columnas y son: 
    ['Loyalty Number', 'Country', 'Province', 'City', 'Postal Code', 'Gender', 'Education', 'Marital Status', 'Loyalty Card', 'CLV', 'Enrollment Type', 'Enrollment Year', 'Enrollment Month']


    A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
Loyalty Number,0.0%,100.0%,int64,16737
Country,0.0%,100.0%,object,1
Province,0.0%,100.0%,object,11
City,0.0%,100.0%,object,29
Postal Code,0.0%,100.0%,object,55


Principales estadísticos de las columnas categóricas:


,count,unique,top,freq
Country,16737,1,Canada,16737
Province,16737,11,Ontario,5404
City,16737,29,Toronto,3351
Postal Code,16737,55,V6E 3D9,911
Gender,16737,2,Female,8410
Education,16737,5,Bachelor,10475
Marital Status,16737,3,Married,9735
Loyalty Card,16737,3,Star,7637
Enrollment Type,16737,2,Standard,15766


Principales estadísticos de las columnas numéricas:


,count,mean,std,min,25%,50%,75%,max
Loyalty Number,16737.0,549735.880445,258912.132453,100018.0,326603.0,550434.0,772019.0,999986.0
Salary,12499.0,79359.340907,34749.691464,9081.0,59246.5,73455.0,88517.5,407228.0
CLV,16737.0,7988.896536,6860.98228,1898.01,3980.84,5780.18,8940.58,83325.38
Enrollment Year,16737.0,2015.253211,1.979111,2012.0,2014.0,2015.0,2017.0,2018.0
Enrollment Month,16737.0,6.669116,3.398958,1.0,4.0,7.0,10.0,12.0
Cancellation Year,2067.0,2016.503145,1.380743,2013.0,2016.0,2017.0,2018.0,2018.0
Cancellation Month,2067.0,6.962748,3.455297,1.0,4.0,7.0,10.0,12.0


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
Loyalty Number,0.0%,100.0%,int64,16737
Country,0.0%,100.0%,object,1
Province,0.0%,100.0%,object,11
City,0.0%,100.0%,object,29
Postal Code,0.0%,100.0%,object,55
Gender,0.0%,100.0%,object,2
Education,0.0%,100.0%,object,5
Salary,25.32%,74.68%,float64,5890
Marital Status,0.0%,100.0%,object,3
Loyalty Card,0.0%,100.0%,object,3


In [8]:
duplicados = df_flight[df_flight.duplicated(keep=False)] 
duplicados.tail(10)

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
404667,949628,2018,12,0,0,0,0,0.0,0,0
404668,949628,2018,12,0,0,0,0,0.0,0,0
404883,960050,2018,12,0,0,0,0,0.0,0,0
404884,960050,2018,12,0,0,0,0,0.0,0,0
405110,971370,2018,12,0,0,0,0,0.0,0,0
405111,971370,2018,12,0,0,0,0,0.0,0,0
405409,988392,2018,12,0,0,0,0,0.0,0,0
405410,988392,2018,12,0,0,0,0,0.0,0,0
405436,989528,2018,12,0,0,0,0,0.0,0,0
405437,989528,2018,12,0,0,0,0,0.0,0,0


In [9]:
df_flight[df_flight["Loyalty Number"] == 208485].head(6)

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
2032,208485,2017,1,4,1,5,2275,227.0,0,0
2033,208485,2017,1,0,0,0,0,0.0,0,0
18933,208485,2017,2,4,0,4,1408,140.0,0,0
18934,208485,2017,2,1,1,2,874,87.0,0,0
35834,208485,2017,3,3,0,3,2208,220.0,0,0
35835,208485,2017,3,0,0,0,0,0.0,0,0


In [10]:
df_flight.columns

Index(['Loyalty Number', 'Year', 'Month', 'Flights Booked',
       'Flights with Companions', 'Total Flights', 'Distance',
       'Points Accumulated', 'Points Redeemed', 'Dollar Cost Points Redeemed'],
      dtype='object')

In [7]:
# agrupo por loyalty number, año y mes, y que me sume el resto de valores para solucionar problema de registros duplicados (meses)
df_flight_agrupado = df_flight.groupby(["Loyalty Number", "Year", "Month"], as_index=False).agg({'Flights Booked': 'sum', 'Flights with Companions': 'sum', 'Total Flights': 'sum', 'Distance': 'sum', 'Points Accumulated': 'sum', 'Points Redeemed': 'sum', 'Dollar Cost Points Redeemed': 'sum'})

In [53]:
df_flight_agrupado.duplicated().sum()

np.int64(0)

In [13]:
df_flight_agrupado.tail()

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
401683,999986,2018,8,8,1,9,3672,367.0,655,53
401684,999986,2018,9,0,0,0,0,0.0,0,0
401685,999986,2018,10,6,0,6,708,70.0,0,0
401686,999986,2018,11,11,4,15,2040,204.0,0,0
401687,999986,2018,12,0,0,0,0,0.0,0,0


In [13]:
df_flight_agrupado[df_flight_agrupado["Loyalty Number"] == 208485]

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
48384,208485,2017,1,4,1,5,2275,227.00,0,0
48385,208485,2017,2,5,1,6,2282,227.00,0,0
48386,208485,2017,3,3,0,3,2208,220.00,0,0
48387,208485,2017,4,12,6,18,1440,144.00,0,0
48388,208485,2017,5,13,0,13,3848,384.00,0,0
48389,208485,2017,6,16,5,21,4137,413.00,0,0
48390,208485,2017,7,20,9,29,2523,252.00,0,0
48391,208485,2017,8,27,0,27,6426,642.00,0,0
48392,208485,2017,9,13,0,13,3497,349.00,0,0
48393,208485,2017,10,19,3,22,4268,426.00,0,0


In [48]:
sp.exploracion(df_flight_agrupado)

El DataFrame tiene 401688 filas y 10 columnas.
    Tiene 0 datos duplicados, lo que supone un porcentaje de 0.0% de los datos.

    Hay 0 columnas con datos nulos, y son: 
    []

    y sin nulos hay 10 columnas y son: 
    ['Loyalty Number', 'Year', 'Month', 'Flights Booked', 'Flights with Companions', 'Total Flights', 'Distance', 'Points Accumulated', 'Points Redeemed', 'Dollar Cost Points Redeemed']


    A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
Loyalty Number,0.0%,100.0%,int64,16737
Year,0.0%,100.0%,int64,2
Month,0.0%,100.0%,int64,12
Flights Booked,0.0%,100.0%,int64,40
Flights with Companions,0.0%,100.0%,int64,21


Principales estadísticos de las columnas categóricas:
No hay columnas categóricas en el DataFrame.
Principales estadísticos de las columnas numéricas:


,count,mean,std,min,25%,50%,75%,max
Loyalty Number,401688.0,549735.880445,258904.719886,100018.0,326603.00,550434.0,772019.00,999986.0
Year,401688.0,2017.500000,0.500001,2017.0,2017.00,2017.5,2018.00,2018.0
Month,401688.0,6.500000,3.452057,1.0,3.75,6.5,9.25,12.0
Flights Booked,401688.0,4.155374,5.269271,0.0,0.00,1.0,8.00,39.0
Flights with Companions,401688.0,1.041916,2.090835,0.0,0.00,0.0,1.00,20.0
Total Flights,401688.0,5.197290,6.576952,0.0,0.00,1.0,10.00,57.0
Distance,401688.0,1220.725451,1446.440549,0.0,0.00,524.0,2352.00,11244.0
Points Accumulated,401688.0,124.904743,147.982107,0.0,0.00,53.0,240.00,1216.5
Points Redeemed,401688.0,30.997660,126.104987,0.0,0.00,0.0,0.00,996.0
Dollar Cost Points Redeemed,401688.0,2.508848,10.200090,0.0,0.00,0.0,0.00,80.0


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
Loyalty Number,0.0%,100.0%,int64,16737
Year,0.0%,100.0%,int64,2
Month,0.0%,100.0%,int64,12
Flights Booked,0.0%,100.0%,int64,40
Flights with Companions,0.0%,100.0%,int64,21
Total Flights,0.0%,100.0%,int64,52
Distance,0.0%,100.0%,int64,5169
Points Accumulated,0.0%,100.0%,float64,1736
Points Redeemed,0.0%,100.0%,int64,591
Dollar Cost Points Redeemed,0.0%,100.0%,int64,52


In [54]:
df_loyalty["Cancellation Year"].unique()

<IntegerArray>
[<NA>, 2018, 2015, 2017, 2014, 2016, 2013]
Length: 7, dtype: Int64

In [8]:
# Convierto los valores de  Cancellation Year y Cancelation month  integer y conservo los nulos
sp.convertir_int(df_loyalty, "Cancellation Year")

In [9]:
sp.convertir_int(df_loyalty, "Cancellation Month")

In [8]:
df_loyalty.head()

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,<NA>,<NA>
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,<NA>,<NA>
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018,1
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,<NA>,<NA>
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,<NA>,<NA>


In [10]:
#Uno los dos dataframes con merge por la columna en comun loyalty number
df_union = df_flight_agrupado.merge(df_loyalty, on = "Loyalty Number")

In [56]:
df_union.head()

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,100018,2017,1,3,0,3,1521,152.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,<NA>,<NA>
1,100018,2017,2,2,2,4,1320,132.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,<NA>,<NA>
2,100018,2017,3,14,3,17,2533,253.0,438,36,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,<NA>,<NA>
3,100018,2017,4,4,0,4,924,92.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,<NA>,<NA>
4,100018,2017,5,0,0,0,0,0.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,<NA>,<NA>


In [17]:
df_union["Loyalty Card"].unique()

array(['Aurora', 'Nova', 'Star'], dtype=object)

In [11]:
df_union.to_csv("data/union_tablas.csv")